In [397]:
import torch
import torch.nn.utils.prune as prune 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go 
import pandas as pd 
import torchvision
from torch.autograd import Variable
import os

In [398]:
check_gpu = torch.cuda.is_available()
check_gpu

True

In [399]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [400]:
check_devices = torch.cuda.get_device_name()
check_devices

'NVIDIA GeForce RTX 2080 Ti'

In [401]:
crypto_name = 'apeusd'
#example_ohlc_df = pd.DataFrame({"Time": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Open": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"High": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Low": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Close": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

ohlc_df = pd.read_csv('historical_price/' + crypto_name + '.csv')
ohlc_df.index = pd.to_datetime(ohlc_df['time'], unit = 'ms')

#ohlc_df.index = ohlc_df.index.tz_localize('UTC').tz_convert('US/Eastern')

In [402]:
price_dataset = ohlc_df.close[-1000:]
time_index = ohlc_df.index[-1000:]
print(time_index[:5])
price_dataset.head()


DatetimeIndex(['2022-06-19 14:14:00', '2022-06-19 14:16:00',
               '2022-06-19 14:17:00', '2022-06-19 14:18:00',
               '2022-06-19 14:19:00'],
              dtype='datetime64[ns]', name='time', freq=None)


time
2022-06-19 14:14:00    3.7809
2022-06-19 14:16:00    3.7974
2022-06-19 14:17:00    3.8011
2022-06-19 14:18:00    3.8081
2022-06-19 14:19:00    3.8141
Name: close, dtype: float64

In [403]:
price_dataset_x = price_dataset[200:]
time_x = time_index[200:]
price_dataset_y = price_dataset[200:400]
time_y = time_index[200:400]
print(len(price_dataset_y))

200


In [404]:
price_test_x = price_dataset[400:600]
time_test_x = time_index[400:600]
price_test_y = price_dataset[600:900]
time_test_y = time_index[600:900]

In [405]:
seq_len = 10
batch_size = 64

In [406]:
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, seq_len=5):
        self.X = torch.tensor(X, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.float32)
        self.seq_len = seq_len
    def __len__(self):
        return self.X.__len__() - (self.seq_len - 1)

    def __getitem__(self, index):
        #print(len(self.y))
        X = self.X[index:index+self.seq_len]
        try:
            y = self.y[index+self.seq_len]
        except:
            y = self.y[self.seq_len]
        return X, y

In [407]:
train_dataset = TimeSeriesDataset(price_dataset_x, price_dataset_y, seq_len=seq_len)
test_dataset = TimeSeriesDataset(price_test_x, price_test_y, seq_len=seq_len)

In [408]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    drop_last = True, 
    shuffle=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    drop_last=True,
    shuffle=False)

In [409]:
class LSTM_Model(torch.nn.Module):
    def __init__(self, input_dim , hidden_size , num_layers, batch_size):
        super(LSTM_Model, self).__init__()
        self.num_layers = num_layers
        self.input_size = input_dim
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.lstm = torch.nn.LSTM(input_size=input_dim , hidden_size = hidden_size , num_layers= num_layers )
        self.fc = torch.nn.Linear(hidden_size,1)

    def forward(self,x,hn,cn):
        out , (hn,cn) = self.lstm(x , (hn,cn))
        final_out = self.fc(out[-1])
        return final_out,hn,cn

    def predict(self,x):
        hn,cn  = self.init()
        final_out = self.fc(out[-1])
        return final_out

    def init(self):
        h0 =  torch.zeros(self.num_layers , self.batch_size , self.hidden_size)
        c0 =  torch.zeros(self.num_layers , self.batch_size , self.hidden_size)
        return h0 , c0


#device = torch.device('cpu')
#print(device)

input_dim = 1
hidden_size = seq_len
num_layers = 3
crypto_price_model = LSTM_Model(input_dim, hidden_size, num_layers, batch_size)


crypto_price_model#.to(device)

loss_function = torch.nn.MSELoss() # Criterion, I believe CrossEntropyLoss has SoftMax built-in so I used it here, this is why my last layer doesn't have a softmax
optimizer  = torch.optim.Adam(crypto_price_model.parameters(), lr=0.017) # used adaptive moment estimation to optimize the model

In [410]:
#training loop
def train(dataloader, model):
    predictions = []
    loss_list = []
    hn , cn = model.init()
    model.train()
    for batch , item in enumerate(dataloader):
        x , y = item
        y = y.type(torch.FloatTensor)
        #x = x.to(device)
        #y = y.to(device)
        out , hn , cn = model(x.reshape(seq_len,batch_size,-1),hn,cn)
        out = out.view(-1)
        # print(out.shape)
        # print(y.shape)
        loss = loss_function(out.reshape(batch_size) , y)
        hn = hn.detach()
        cn = cn.detach()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"train loss: {loss:>7f} ")
        loss_list.append(loss)
        predictions.append(out.detach().numpy())
        return predictions, loss_list
        

In [411]:
def test(dataloader, model):
    predictions = []
    loss_list = []
    hn , cn = model.init()
    model.eval()
    for batch , item in enumerate(dataloader):
        x , y = item
        y = y.type(torch.FloatTensor)
        #x = x.to(device)
        #y = y.to(device)
        out , hn , cn = model(x.reshape(seq_len,batch_size,1),hn,cn)
        loss = loss_function(out.reshape(batch_size) , y)
       
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"test loss: {loss:>7f} ")
        predictions.append(out.detach().numpy())
        loss_list.append(loss)
        return predictions, loss_list

In [412]:
epochs = 100
for epoch in range(epochs):
    print(f"epoch {epoch} ")
    train(train_loader, crypto_price_model)
    test_predictions = test(test_loader, crypto_price_model)
#print(test_predictions[:][:][:][0])

epoch 0 
epoch 1 
epoch 2 
epoch 3 
epoch 4 
epoch 5 
epoch 6 
epoch 7 
epoch 8 
epoch 9 
epoch 10 
epoch 11 
epoch 12 
epoch 13 
epoch 14 
epoch 15 
epoch 16 
epoch 17 
epoch 18 
epoch 19 
epoch 20 
epoch 21 
epoch 22 
epoch 23 
epoch 24 
epoch 25 
epoch 26 
epoch 27 
epoch 28 
epoch 29 
epoch 30 
epoch 31 
epoch 32 
epoch 33 
epoch 34 
epoch 35 
epoch 36 
epoch 37 
epoch 38 
epoch 39 
epoch 40 
epoch 41 
epoch 42 
epoch 43 
epoch 44 
epoch 45 
epoch 46 
epoch 47 
epoch 48 
epoch 49 
epoch 50 
epoch 51 
epoch 52 
epoch 53 
epoch 54 
epoch 55 
epoch 56 
epoch 57 
epoch 58 
epoch 59 
epoch 60 
epoch 61 
epoch 62 
epoch 63 
epoch 64 
epoch 65 
epoch 66 
epoch 67 
epoch 68 
epoch 69 
epoch 70 
epoch 71 
epoch 72 
epoch 73 
epoch 74 
epoch 75 
epoch 76 
epoch 77 
epoch 78 
epoch 79 
epoch 80 
epoch 81 
epoch 82 
epoch 83 
epoch 84 
epoch 85 
epoch 86 
epoch 87 
epoch 88 
epoch 89 
epoch 90 
epoch 91 
epoch 92 
epoch 93 
epoch 94 
epoch 95 
epoch 96 
epoch 97 
epoch 98 
epoch 99 
[4.017255 

In [416]:
fp = np.array([num for num in test_predictions[0][0]]).flatten()
print(fp)

[4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255
 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255
 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255
 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255
 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255
 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255
 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255
 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255 4.017255]


In [413]:
print(len(test_predictions[0]))
print()
fig = go.Figure()
pred_line = go.Scatter(x=time_test_y[:len(fp)], y=fp, mode = 'lines', name = 'Predicted Price')
fig.add_trace(pred_line)

fig.add_trace(go.Scatter(x=time_test_y[:len(fp)], y=price_test_y[:len(fp)], mode = 'lines', name = 'Actual Price'))

fig.update_layout(xaxis_title="Time", yaxis_title="Actual Price ($)", title=f"Predicted Price vs Actual Price for '{crypto_name}'")

fig.show()

1



In [414]:
torch.save(crypto_price_model.state_dict(), f'{crypto_name}_model_weights.pth')

In [415]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
crypto_price_model.load_state_dict(torch.load(f'{crypto_name}_model_weights.pth', map_location=torch.device(device)))

cuda


RuntimeError: CUDA error: device-side assert triggered